In [3]:
from math import sin
from random import random, uniform
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from math import e, sqrt,cos,pi

In [4]:
class Point():
    def __init__(self, point, pheromone) -> None:
        self.point = point
        self.pheromone = pheromone
    
    def get_point(self):
        return self.point
    
    def get_pheromone(self):
        return self.pheromone

    def set_pheromone(self, pheromone):
        self.pheromone = pheromone
    
    def set_point(self, point):
        self.point = point

    def __str__(self):
        return "point: " + str(self.point) + "," + "pheromone: " + str(self.pheromone)

class Ant():

    def __init__(self, memory_limit) -> None:
        self.memory = list()
        self.memory_limit = memory_limit
        self.current_localization = list()

    def clear_location(self):
        self.current_localization = list()

    def get_location(self):
        output_list = list()
        for point in self.current_localization:
            output_list.append(point.get_point())
        return output_list

    def update_location(self, new_location):
        for i in range(len(self.current_localization)):
            self.current_localization[i].set_point(new_location[i])

    def assign_point(self, point):
        self.current_localization.append(point)

    def update_pheromone(self, error):
        for point in self.current_localization:
            point.set_pheromone(point.get_pheromone() + (1/error))

    def set_memory(self, point):
        for p in self.memory:
            if(point.get_point() == p.get_point()): 
                return False
        self.memory.append(point)
        if( len(self.memory) > self.memory_limit ):
            del self.memory[0]
        return True
    
    def get_i_point(self,i):
        return self.memory[i]

    def __str__(self):
        memory = ""
        for point in self.memory:
            memory += " " + str(point) + " "
        location = ""
        for point in self.current_localization:
            location += " " + str(point) + " "
        return "memory: " + memory + " and " + "current location" + location

class PointsList():
    def __init__(self, list_of_points) -> None:
        self.points = list_of_points
    
    def get_best_point(self):
        best_point = Point(0,0)
        for point in self.points:
            if(point.get_pheromone() > best_point.get_pheromone()):
                best_point = point
        return best_point

    def get_total_pheromones(self):
        total = 0
        for point in self.points:
            total += point.get_pheromone()
        return total

    def get_list_points(self):
        return self.points

    def get_i_point(self, i):
        return self.points[i]

    def evaporate_pheromone(self, p):
        for point in self.points:
            point.set_pheromone((1-p)*point.get_pheromone())

class ACO():

    def __init__(self, num_params, discrete_points, interval, number_ants, q, evaporation_rate, num_iterations = 50) -> None:
        self.number_params = num_params
        self.num_iterations = num_iterations
        self.discrete_points = discrete_points
        self.points = list()
        self.q = q
        self.p = evaporation_rate
        self.ants = [Ant(num_params) for _ in range(0, number_ants)]
        for _ in range(0,self.number_params):
            self.points.append(PointsList([Point(uniform(interval[0],interval[1]), 1/2) for _ in range(discrete_points)]))

    def get_best_ant(self, function):
        cost = 0
        best_ant = self.ants[0]
        for ant in self.ants:
            ant_cost = -1 * (function(ant.get_location()))
            if(ant_cost < cost):
                cost = ant_cost
                best_ant = ant
        return best_ant, cost

    def local_search(self, function):
        for ant in self.ants:
            res = minimize(function, ant.get_location(), method='COBYLA')
            ant.update_location(res.x)

        
    def update_pheromone(self, ant, cost):
        ant.update_pheromone(cost)
        for point_list in self.points:
            point_list.evaporate_pheromone(self.p)


    def probabilistic_construction(self):
        for ant in self.ants:
            ant.clear_location()
            if(random() > 1 - self.q):
                for point_list in self.points:
                    ant_asigned = ant.set_memory(point_list.get_best_point())
                    ant.assign_point(point_list.get_best_point())
            else:
                for point_list in self.points:
                    for point in point_list.get_list_points():
                        if(random() > (point.get_pheromone())/point_list.get_total_pheromones()):
                            ant_asigned = ant.set_memory(point)
                            if (ant_asigned):
                                ant.assign_point(point)
                                break

    def run(self,fx):
        aco.probabilistic_construction()
        aco.local_search(fx)
        best_ant, best_cost = aco.get_best_ant(fx)
        if(best_cost == 0):
            print("solution found")
            return ant.get_location()
        else:
            aco.update_pheromone(best_ant, -1*best_cost)
        for i in range(self.num_iterations):
            aco.probabilistic_construction()
            aco.local_search(fx)
            ant, cost = aco.get_best_ant(fx)
            if(cost == 0):
                print("solution found")
                return ant.get_location()
            else:
                aco.update_pheromone(ant, -1*cost)
                if(i % 25 == 0):
                    print("iteration " + str(i) + " with cost of " + str(-1*cost))
            if(cost < best_cost):
                best_ant = ant
                best_cost = cost
        return best_ant.get_location(),best_cost 
                        

## TEST'S

### Himmelblau's function

![alt text](him.png "Title")

In [219]:
fx = lambda x : (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 -7)**2

In [203]:
minimize(fx, [uniform(-5,5),uniform(-5,5)], method='COBYLA')

     fun: 2.648055705595566e-07
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 44
  status: 1
 success: True
       x: array([ 3.58449999, -1.84814548])

In [220]:
aco = ACO(num_params=2,discrete_points=100,interval=[-5,5],
number_ants=20,q=0.01, evaporation_rate=0.9, num_iterations = 100)
local, cost = aco.run(fx)
print("The ant location was: " + str(local))

iteration 0 with cost of 5.655259173415215e-07
iteration 25 with cost of 34.9812997571636
iteration 50 with cost of 4.150933615619393e-07
iteration 75 with cost of 4.557373861387135e-07
The ant location was: [2.999994508708392, 1.9999901130873414]


### Ackley function

![alt text](akley.png "Title")

In [221]:
fx = lambda x : -20*e**(-0.2*sqrt(0.5*(x[0]**2 + x[1]**2)))-e**(0.5*(cos(2*pi*x[1])+cos(2*pi*x[0])))+e+20

In [201]:
minimize(fx, [uniform(-5,5),uniform(-5,5)], method='COBYLA')

     fun: 0.0003354255648062576
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 44
  status: 1
 success: True
       x: array([-4.31722953e-05,  1.10311491e-04])

In [222]:
aco = ACO(num_params=2,discrete_points=100,interval=[-5,5],
number_ants=20,q=0.01, evaporation_rate=0.9, num_iterations = 100)
local, cost = aco.run(fx)
print("The ant location was: " + str(local))

iteration 0 with cost of 0.0001793948011794555
iteration 25 with cost of 0.0004967078082778187
iteration 50 with cost of 0.0004038308119049816
iteration 75 with cost of 0.0002325746366764747
The ant location was: [8.102717413325142e-06, -0.0001279816775409477]


### Rastrigin

![alt text](ras.png "Title")

In [7]:
fx = lambda x : 10*2 + (x[0]**2 - 10*cos(2*pi*x[0])) + (x[1]**2 - 10*cos(2*pi*x[1]))

In [8]:
minimize(fx, [uniform(-5,5),uniform(-5,5)], method='COBYLA')

     fun: 0.9949618630896477
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 41
  status: 1
 success: True
       x: array([-9.56470955e-05, -9.95029333e-01])

In [6]:
aco = ACO(num_params=2,discrete_points=100,interval=[-5,5],
number_ants=20,q=0.01, evaporation_rate=0.9, num_iterations = 100)
local, cost = aco.run(fx)
print("The ant location was: " + str(local))

iteration 0 with cost of 3.979831388054432
iteration 25 with cost of 3.9798349026176734
iteration 50 with cost of 3.9798313086838206
iteration 75 with cost of 3.9798328401588368
The ant location was: [3.374949091509218e-05, -5.7081542194723354e-05]
